---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    universityTownsRaw = pd.read_table('university_towns.txt', header=None)
    universityTowns = pd.DataFrame(None, columns=['State', 'RegionName'])
    for i in universityTownsRaw[0]:
        if (i[-6:] == '[edit]' and i[:-6] in states.values()):
            j = i[:-6]
        elif '(' in i:
            universityTowns = universityTowns.append({'State': j, 'RegionName': i[:i.index('(')-1]}, ignore_index=True)
        else:
            universityTowns = universityTowns.append({'State': j, 'RegionName': i}, ignore_index=True)
    return universityTowns
get_list_of_university_towns()

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    GDPRaw = pd.read_excel('gdplev.xls')
    GDP = GDPRaw[GDPRaw.columns[[4, 5, 6]]].rename(columns={'Unnamed: 4': 'Quarter', 'Unnamed: 5': 'GDP in billions of current dollars', \
                                                            'Unnamed: 6': 'GDP in billions of chained 2009 dollars'}).drop(GDPRaw.index[0:219]).reset_index(drop=True)
    recessionStart = ''
    for i in range(len(GDP)):
        try:
            if (GDP['GDP in billions of chained 2009 dollars'][i] > GDP['GDP in billions of chained 2009 dollars'][i + 1] and \
                GDP['GDP in billions of chained 2009 dollars'][i + 1] > GDP['GDP in billions of chained 2009 dollars'][i + 2]):
                    recessionStart = GDP['Quarter'][i+1]
                    break
        except KeyError:
            pass
    return recessionStart
get_recession_start()

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    GDPRaw = pd.read_excel('gdplev.xls')
    GDP = GDPRaw[GDPRaw.columns[[4, 5, 6]]].rename(columns={'Unnamed: 4': 'Quarter', 'Unnamed: 5': 'GDP in billions of current dollars', \
                                                            'Unnamed: 6': 'GDP in billions of chained 2009 dollars'}).drop(GDPRaw.index[0:219]).reset_index(drop=True)
    for i in range(len(GDP)):
        try:
            if GDP['GDP in billions of chained 2009 dollars'][i] > GDP['GDP in billions of chained 2009 dollars'][i + 1]:
                if GDP['GDP in billions of chained 2009 dollars'][i + 1] > GDP['GDP in billions of chained 2009 dollars'][i + 2]:
                    j = 0
                    k = 2
                    l = 3
                    m = 2
                    while j < 2:
                        if GDP['GDP in billions of chained 2009 dollars'][i + k] < GDP['GDP in billions of chained 2009 dollars'][i + l]:
                            j += 1
                        k += 1
                        l += 1
                        m += 1
                    recessionEnd = GDP['Quarter'][i + m]
        except KeyError:
            pass
    return recessionEnd
get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    GDPRaw = pd.read_excel('gdplev.xls')
    GDP = GDPRaw[GDPRaw.columns[[4, 5, 6]]].rename(columns={'Unnamed: 4': 'Quarter', 'Unnamed: 5': 'GDP in billions of current dollars', \
                                                            'Unnamed: 6': 'GDP in billions of chained 2009 dollars'}).drop(GDPRaw.index[0:219]).reset_index(drop=True)
    recessionStart = get_recession_start()
    recessionEnd = get_recession_end()
    for i in range(len(GDP)):
        if GDP['Quarter'][i] == recessionStart:
            j = i
        if GDP['Quarter'][i] == recessionEnd:
            k = i
    for l in range(len(GDP)):
        if GDP['GDP in billions of chained 2009 dollars'][l] == GDP['GDP in billions of chained 2009 dollars'][j:k].min():
            recessionBottom = GDP['Quarter'][l]
    return recessionBottom
get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    cityAllHomesRaw = pd.read_csv('City_Zhvi_AllHomes.csv')
    cityAllHomes = cityAllHomesRaw
    i = 0
    j = 0
    for k in range(51, 251):
        try:
            if int(cityAllHomes.columns[k].split('-0')[1]) <= 3:
                quarter = cityAllHomes.columns[k].split('-')[0] + 'q1'
            if int(cityAllHomes.columns[k].split('-0')[1]) > 3 and int(cityAllHomes.columns[k].split('-0')[1]) <= 6:
                quarter = cityAllHomes.columns[k].split('-')[0] + 'q2'
            if int(cityAllHomes.columns[k].split('-0')[1]) > 6  and int(cityAllHomes.columns[k].split('-0')[1]) <= 8:
                quarter = cityAllHomes.columns[k].split('-')[0] + 'q3'
        except:
            if int(cityAllHomes.columns[k].split('-')[1]) > 8 and int(cityAllHomes.columns[k].split('-')[1]) <= 12:
                quarter = cityAllHomes.columns[k].split('-')[0] + 'q4'
        try:
            cityAllHomes[quarter] = cityAllHomes[quarter] + cityAllHomes[cityAllHomes.columns[k]]
        except:
            cityAllHomes[quarter] = cityAllHomes[cityAllHomes.columns[k]]
        i += 1
        if i == 3:
            cityAllHomes[quarter] = cityAllHomes[quarter] / 3
            i = 0
        if quarter == '2016q3':
            j += 1
            if j == 2:
                cityAllHomes[quarter] = cityAllHomes[quarter] / 2
    cityAllHomes['State'].replace(states, inplace=True)
    cityAllHomes = cityAllHomes[['State', 'RegionName', '2000q1', '2000q2', '2000q3', '2000q4', '2001q1', \
                                 '2001q2', '2001q3', '2001q4', '2002q1', '2002q2', '2002q3', '2002q4', '2003q1', \
                                 '2003q2', '2003q3', '2003q4', '2004q1', '2004q2', '2004q3', '2004q4', '2005q1', \
                                 '2005q2', '2005q3', '2005q4', '2006q1', '2006q2', '2006q3', '2006q4', '2007q1', \
                                 '2007q2', '2007q3', '2007q4', '2008q1', '2008q2', '2008q3', '2008q4', '2009q1', \
                                 '2009q2', '2009q3', '2009q4', '2010q1', '2010q2', '2010q3', '2010q4', '2011q1', \
                                 '2011q2', '2011q3', '2011q4', '2012q1', '2012q2', '2012q3', '2012q4', '2013q1', \
                                 '2013q2', '2013q3', '2013q4', '2014q1', '2014q2', '2014q3', '2014q4', '2015q1', \
                                 '2015q2', '2015q3', '2015q4', '2016q1','2016q2', '2016q3']].set_index(['State', 'RegionName']).sort_index()
    return cityAllHomes
convert_housing_data_to_quarters()

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    universityTowns = get_list_of_university_towns()
    recessionStart = get_recession_start()
    recessionBottom = get_recession_bottom()
    cityAllHomes = convert_housing_data_to_quarters()
    cityAllHomes = cityAllHomes.reset_index().set_index('RegionName')
    cityAllHomes = cityAllHomes.loc[:, [recessionStart, recessionBottom]]
    cityAllHomes['UniversityTown'] = 0
    for i in list(universityTowns['RegionName']):
        if i in cityAllHomes.index.values:
            cityAllHomes['UniversityTown'][i] = 1
    cityAllHomes['PriceRatio'] = (cityAllHomes[recessionStart] - cityAllHomes[recessionBottom]) / cityAllHomes[recessionStart]
    std, p = ttest_ind(cityAllHomes[cityAllHomes['UniversityTown'] == 1]['PriceRatio'], cityAllHomes[cityAllHomes['UniversityTown'] == 0]['PriceRatio'], nan_policy='omit')
    different = False
    if p < 0.01:
        different = True
    if cityAllHomes[cityAllHomes['UniversityTown'] == 1]['PriceRatio'].mean() < cityAllHomes[cityAllHomes['UniversityTown'] == 0]['PriceRatio'].mean():
        meanPriceRatio = 'university town'
    elif cityAllHomes[cityAllHomes['UniversityTown'] == 1]['PriceRatio'].mean() > cityAllHomes[cityAllHomes['UniversityTown'] == 0]['PriceRatio'].mean():
        meanPriceRatio = 'non-university town'
    return (different, p, meanPriceRatio)
run_ttest()